In [1]:
import os
from itertools import chain
from operator import itemgetter
from pprint import pprint

import boto3
from dotenv import load_dotenv
from IPython.display import display
from mypy_boto3_sts.client import GetCallerIdentityResponseTypeDef

load_dotenv()

True

In [2]:
sts = boto3.client("sts")
AWS_ACCOUNT_ID = sts.get_caller_identity()["Account"]

session = boto3.Session()

## S3


In [3]:
SRC_BUCKET = "bucket-dummy-data1"
DST_BUCKET = "your-destination-bucket"
SRC_REGION = "ap-southeast-1"
DST_REGION = "ap-southeast-3"

In [4]:
s3_src = session.client("s3", region_name=SRC_REGION)
s3_dst = session.client("s3", region_name=DST_REGION)

In [5]:
objects = s3_src.list_objects_v2(Bucket=SRC_BUCKET)
if "Contents" in objects:
    for obj in objects["Contents"]:
        if "Key" in obj:
            copy_source = {"Bucket": SRC_BUCKET, "Key": obj["Key"]}

## Glue


In [6]:
def get_db_generator(glue_obj):
    return map(itemgetter("Name"), glue_obj.get_databases().get("DatabaseList"))


def get_table_generator(glue_obj, db_list):
    return map(
        lambda x: {
            x: list(
                map(
                    itemgetter("Name"), glue_obj.get_tables(DatabaseName=x)["TableList"]
                )
            )
        },
        db_list,
    )


In [7]:
glue_src = session.client("glue", region_name=SRC_REGION)
glue_dst = session.client("glue", region_name=DST_REGION)

In [8]:
db_src_list = list(get_db_generator(glue_src))
db_dst_list = list(get_db_generator(glue_dst))

display(db_src_list)
display(db_dst_list)


[]

['database-dummy',
 'database-dummy2',
 'database-dummy3',
 'databases-jkt',
 'databases-sg',
 'netflix_db',
 'schedule-db-sg',
 'test-db',
 'transaction']

In [9]:
db_table_src_list = list(get_table_generator(glue_src, db_src_list))
db_table_dst_list = list(get_table_generator(glue_dst, db_dst_list))

display(db_table_src_list)
display(db_table_dst_list)

[]

[{'database-dummy': ['052315ff_78a0_4874_a8fc_17a68dc39411_csv',
   '052315ff_78a0_4874_a8fc_17a68dc39411_csv_metadata',
   '21',
   '713b0fd1_28fa_4fbc_93bb_dbedeb904743_csv',
   '713b0fd1_28fa_4fbc_93bb_dbedeb904743_csv_metadata',
   'data_dummy_csv',
   'data_dummy_test_fahmi_csv',
   'data_testing_json',
   'f77adf97_2e68_4c46_939b_b4441177e2f0_csv',
   'f77adf97_2e68_4c46_939b_b4441177e2f0_csv_metadata',
   'fahmi-test-2data_testing_json',
   'fahmi-test-data_dummy_csv']},
 {'database-dummy2': []},
 {'database-dummy3': []},
 {'databases-jkt': ['data_dummy_csv',
   'netflix_data',
   'query_test',
   'transaction',
   'transaction_csv',
   'transaction_xlsx',
   'unsaved',
   'web_and_social_analytics_csv']},
 {'databases-sg': ['data_dummy_csv',
   'netflix_data',
   'query_test',
   'transaction',
   'transaction_csv',
   'transaction_xlsx',
   'unsaved',
   'web_and_social_analytics_csv']},
 {'netflix_db': ['netflix_data']},
 {'schedule-db-sg': ['data_dummy_csv',
   'netflix_data